In [95]:
from paddleocr import PaddleOCR, draw_ocr

# Paddleocr supports Chinese, English, French, German, Korean and Japanese.
# You can set the parameter `lang` as `ch`, `en`, `fr`, `german`, `korean`, `japan`
# to switch the language model in order.
ocr_model = PaddleOCR(
    use_angle_cls=True, lang="en", use_gpu=False
)  # need to run only once to download and load model into memory
file_path = "aeg2.pdf"
result = ocr_model.ocr(file_path, cls=True)
# for idx in range(len(result)):
#     res = result[idx]
# for line in res:
#         print('////////////////////////////////////////////////')
#         print(line)

#     print('----------')
#     print(res)
# print('================================================')
# result

[2023/07/20 16:07:59] ppocr DEBUG: Namespace(help='==SUPPRESS==', use_gpu=False, use_xpu=False, use_npu=False, ir_optim=True, use_tensorrt=False, min_subgraph_size=15, precision='fp32', gpu_mem=500, image_dir=None, page_num=0, det_algorithm='DB', det_model_dir='C:\\Users\\Payoff/.paddleocr/whl\\det\\en\\en_PP-OCRv3_det_infer', det_limit_side_len=960, det_limit_type='max', det_box_type='quad', det_db_thresh=0.3, det_db_box_thresh=0.6, det_db_unclip_ratio=1.5, max_batch_size=10, use_dilation=False, det_db_score_mode='fast', det_east_score_thresh=0.8, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_sast_score_thresh=0.5, det_sast_nms_thresh=0.2, det_pse_thresh=0, det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, scales=[8, 16, 32], alpha=1.0, beta=1.0, fourier_degree=5, rec_algorithm='SVTR_LCNet', rec_model_dir='C:\\Users\\Payoff/.paddleocr/whl\\rec\\en\\en_PP-OCRv3_rec_infer', rec_image_inverse=True, rec_image_shape='3, 48, 320', rec_batch_num=6, max_text_length=25, 

[2023/07/20 16:08:01] ppocr DEBUG: dt_boxes num : 96, elapse : 0.6685736179351807
[2023/07/20 16:08:02] ppocr DEBUG: cls num  : 96, elapse : 1.058638572692871
[2023/07/20 16:08:18] ppocr DEBUG: rec_res num  : 96, elapse : 16.43259859085083


In [98]:
# for line in result[0]:
#     print(line[1][0])
#     print('--------------------------------')


allTheTexts = {line[1][0] for line in result[0]}

{' 05/18/2023',
 '$ 325,203.55',
 '-VA-',
 '05/11/2023',
 '1',
 '1 05/01/2023',
 '1585245',
 '19,522.23',
 '2 05/02/2023',
 '21019 SOMMA LOMBARDO',
 '23,229.37',
 '235543',
 '235618',
 '235917',
 '236077',
 '28,611.660',
 '2813634141',
 '3 05/06/2023',
 '3,011.290',
 '325,203.55',
 '4 05/08/2023',
 '45,836.42',
 '5.393258',
 '51990',
 '6.393258',
 '60,615.53',
 '7,081.390',
 '701 Waterford Way, Suite 490, Miami, FL 33126, USA.',
 '8,996.110',
 '8.323379',
 '8686 New Trails Drive',
 '9,522.870',
 '9.393258',
 'ASSOCIATED ENERGY GROUP, LLC',
 'AVIATION TURBINE FUEL - VRMM',
 'Airport: VRMM -MALE,MV',
 'BILL TO:',
 'Comments:',
 'Currency',
 'Customer #',
 'Data : 23/05/2023',
 'Due Date',
 'EINEO',
 'EINEU',
 'EIXIN',
 'Equip. ID',
 'Extended Price',
 'Flight #',
 'Invoice',
 'Invoice #',
 'Invoice Date',
 'Item Desc hk',
 'Item Sub Total',
 'JESSICA RIPAMONTI',
 'LOMBARDY IT 21019',
 'NEOS SPA',
 'NET7',
 'NO141',
 'NO511',
 'NO711',
 'No Trans.Date',
 'PAYMENT DISCLAIMER: To maintain s

In [55]:
import csv

with open("ocr_result.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["Text", "Confidence"])
    for line in result:
        writer.writerow(line)

In [56]:
import json

json_result = []
for line in result:
    json_result.append(line)

    # json_result.append({
    #     'text': line[1][0],
    #     'confidence': line[1][1]
    # })

with open("ocr_result.json", "w") as f:
    json.dump(json_result, f)

In [57]:
# draw result
import fitz
from PIL import Image
import cv2
import numpy as np

imgs = []
with fitz.open(file_path) as pdf:
    for pg in range(0, pdf.pageCount):
        page = pdf[pg]
        mat = fitz.Matrix(2, 2)
        pm = page.get_pixmap(matrix=mat, alpha=False)
        # if width or height > 2000 pixels, don't enlarge the image
        if pm.width > 2000 or pm.height > 2000:
            pm = page.get_pixmap(matrix=fitz.Matrix(1, 1), alpha=False)

        img = Image.frombytes("RGB", [pm.width, pm.height], pm.samples)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        imgs.append(img)

for idx in range(len(result)):
    res = result[idx]
    image = imgs[idx]

    boxes = [line[0] for line in res]
    txts = [line[1][0] for line in res]
    scores = [line[1][1] for line in res]

    # Draw OCR results on image
    im_show = draw_ocr(image, boxes, txts, scores, font_path="simfang.ttf")

    # Convert PIL image to OpenCV format
    im_show = cv2.cvtColor(np.array(im_show), cv2.COLOR_RGB2BGR)

    # Display image
    cv2.imshow("Image", im_show)
    cv2.waitKey(0)

    # Save image
    cv2.imwrite(f"result_page_{file_path}_{idx}.jpg", im_show)

cv2.destroyAllWindows()